In [2]:
import dimers_util
from dimers_util import *
import dimers_sim
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import matplotlib.pyplot as plt
import matplotlib
from importlib import reload 
reload(dimers_util)
reload(dimers_sim)

<module 'dimers_sim' from 'C:\\Users\\ofir\\OneDrive - Technion\\Research - PhD\\DynamicalConfinement\\DymerKrylov\\dimers_sim.py'>

In [ ]:
d_sim = [20, 30]
L_sim = 40
times_sim = 400
nums_sim = 60000
simulator = dimers_sim.Simulator(L_sim, times_sim, d_sim, nums_sim, d_procs_num=2, nums_subprocs_num=3)
r = simulator.parallel_analysis()

38 220
Waiting for all processes to close
0 items waiting


In [ ]:
dimers_sim.plot_analysis(simulator.analysis_rhos, L_sim, times_sim, nums_sim)

In [9]:
# numpy.apply_along_axis
H_ring, H_hopp = get_h_ring(L), get_h_hop(L)
gates = H_ring + H_hopp
p_array = np.concatenate((np.ones(len(H_ring)), np.ones(len(H_hopp))))/(len(H_ring)+len(H_hopp))
psi =  np.array([get_initial_config(10, 5) for i in range(3)])
gates_i = np.random.choice(gates, size=psi.shape[0], p=p_array)

def apply(f, a):
    return f(a)
psi = np.array(map(apply, zip(psi, gates_i))
print(psi)

SyntaxError: invalid syntax (3941479316.py, line 11)